# Module 3 Assignment: Analyzing and Mitigating Bias in Machine Learning

## Dataset: UCI Adult Income Dataset

This notebook analyzes fairness and bias in machine learning using the Adult Income dataset.

## Assignment Steps:
1. Data Preprocessing and Model Training
2. Calculate Fairness Metrics (Demographic Parity, Equalized Odds)
3. Mitigation Technique Implementation (Reweighting)
4. Report Findings


## 1. Data Preprocessing and Model Training


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing



In [4]:
# Load the dataset
df = pd.read_csv('adult_raw/adult_raw.csv')

print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
df.head()


Dataset shape: (32561, 15)

Column names:
['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

First few rows:


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
# Basic preprocessing, note that column names may need to be adjusted depending on
# the datset's structure. 
df['sex'] = df['sex'].str.strip().apply(lambda x: 1 if x == 'Male' else 0)


df = pd.get_dummies(df, columns=["workclass", "education", "marital-status", "occupation", "relationship", "race", "native-country"], drop_first=True)
df['income'] = df['income'].str.strip().apply(lambda x: 1 if x == ">50K" else 0)

print("Processed dataset shape:", df.shape)
print("\nProcessed columns:")
print(df.columns.tolist())


Processed dataset shape: (32561, 101)

Processed columns:
['age', 'fnlwgt', 'education-num', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'income', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'marital-status_ Married-AF-spouse', 'marital-status_ Married-civ-spouse', 'marital-status_ Married-spouse-absent', 'marital-status_ Never-married', 'marital-status_ Separated', 'marital-status_ Widowed', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ Craft-repair', 'occ

In [6]:
# Train/test split 
X = df.drop("income", axis=1)
y = df["income"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Training set income distribution: {y_train.value_counts(normalize=True)}")
print(f"Test set income distribution: {y_test.value_counts(normalize=True)}")


Training set size: 22792
Test set size: 9769
Training set income distribution: income
0    0.757503
1    0.242497
Name: proportion, dtype: float64
Test set income distribution: income
0    0.763128
1    0.236872
Name: proportion, dtype: float64


In [7]:
# Train logistic regression model 
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




Accuracy: 0.8006960794349472


c:\Users\vp431\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 2. Calculate Fairness Metrics 


In [8]:
# Create StandardDataset 
dataset = StandardDataset(
    df,
    label_name="income",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]]
)

print("Dataset created for analysis")
print(f"Dataset shape: {dataset.features.shape}")
print(f"Label distribution: {np.bincount(dataset.labels.ravel().astype(int))}")

# Calculate baseline fairness metrics
dataset_metric = BinaryLabelDatasetMetric(
    dataset,
    unprivileged_groups=[{'sex': 0}],
    privileged_groups=[{'sex': 1}]
)

print("\n=== BASELINE FAIRNESS METRICS ===")
print(f"Demographic Parity Difference: {dataset_metric.statistical_parity_difference():.4f}")
print(f"Disparate Impact: {dataset_metric.disparate_impact():.4f}")


Dataset created for analysis
Dataset shape: (32561, 100)
Label distribution: [24720  7841]

=== BASELINE FAIRNESS METRICS ===
Demographic Parity Difference: -0.1963
Disparate Impact: 0.3580


In [9]:

classification_metric = ClassificationMetric(dataset, dataset, 
                                           unprivileged_groups=[{'sex': 0}], 
                                           privileged_groups=[{'sex': 1}])

print("\n=== CLASSIFICATION METRICS ===")
print(f"True Positive Rate: {classification_metric.true_positive_rate():.4f}")
print(f"False Positive Rate: {classification_metric.false_positive_rate():.4f}")
print(f"True Negative Rate: {classification_metric.true_negative_rate():.4f}")
print(f"False Negative Rate: {classification_metric.false_negative_rate():.4f}")
print(f"Accuracy: {classification_metric.accuracy():.4f}")



=== CLASSIFICATION METRICS ===
True Positive Rate: 1.0000
False Positive Rate: 0.0000
True Negative Rate: 1.0000
False Negative Rate: 0.0000
Accuracy: 1.0000


In [10]:
# Store baseline metrics for comparison
baseline_dp_diff = dataset_metric.statistical_parity_difference()
baseline_accuracy = accuracy

print(f"\nBaseline metrics stored:")
print(f"Demographic Parity Difference: {baseline_dp_diff:.4f}")
print(f"Accuracy: {baseline_accuracy:.4f}")



Baseline metrics stored:
Demographic Parity Difference: -0.1963
Accuracy: 0.8007


## 3. Mitigation Technique Implementation (Reweighting)


In [11]:
# Implement reweighting 
RW = Reweighing(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
dataset_transf = RW.fit_transform(dataset)

print("Reweighting applied using AIF360")
print(f"Transformed dataset shape: {dataset_transf.features.shape}")

# Calculate fairness metrics after reweighting
dataset_transf_metric = BinaryLabelDatasetMetric(dataset_transf, 
                                                unprivileged_groups=[{'sex': 0}], 
                                                privileged_groups=[{'sex': 1}])

print("\n=== FAIRNESS METRICS AFTER REWEIGHTING ===")
print(f"Demographic Parity Difference: {dataset_transf_metric.statistical_parity_difference():.4f}")
print(f"Disparate Impact: {dataset_transf_metric.disparate_impact():.4f}")


Reweighting applied using AIF360
Transformed dataset shape: (32561, 100)

=== FAIRNESS METRICS AFTER REWEIGHTING ===
Demographic Parity Difference: 0.0000
Disparate Impact: 1.0000


In [12]:
# Re-train and evaluate fairness again with the reweighted dataset
# Extract features and labels from transformed dataset
X_transf = dataset_transf.features
y_transf = dataset_transf.labels.ravel()

# Split the transformed dataset
X_train_transf, X_test_transf, y_train_transf, y_test_transf = train_test_split(
    X_transf, y_transf, test_size=0.3, random_state=42)

# Train model on reweighted data
model_transf = LogisticRegression()
model_transf.fit(X_train_transf, y_train_transf)
y_pred_transf = model_transf.predict(X_test_transf)

# Evaluate reweighted model performance
accuracy_transf = accuracy_score(y_test_transf, y_pred_transf)
print(f"\nReweighted Model Accuracy: {accuracy_transf:.4f}")
print(f"Accuracy Change: {accuracy_transf - baseline_accuracy:.4f}")





Reweighted Model Accuracy: 0.8007
Accuracy Change: 0.0000


c:\Users\vp431\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
predicted_dataset = dataset.copy()
predicted_dataset.labels = y_pred.reshape(-1, 1)

classification_metric = ClassificationMetric(
    dataset, predicted_dataset,
    unprivileged_groups=[{'sex': 0}],
    privileged_groups=[{'sex': 1}]
)



## 4. Report Findings


In [14]:
# Compare fairness metrics before and after mitigation
print("\n" + "="*60)
print("FAIRNESS COMPARISON: BEFORE vs AFTER MITIGATION")
print("="*60)

# Store reweighted metrics
reweighted_dp_diff = dataset_transf_metric.statistical_parity_difference()

print(f"\nMODEL PERFORMANCE:")
print(f"Baseline Accuracy: {baseline_accuracy:.4f}")
print(f"Reweighted Accuracy: {accuracy_transf:.4f}")
print(f"Accuracy Change: {accuracy_transf - baseline_accuracy:.4f}")

print(f"\nFAIRNESS METRICS:")
print(f"Demographic Parity Difference:")
print(f"  Before: {baseline_dp_diff:.4f}")
print(f"  After:  {reweighted_dp_diff:.4f}")
print(f"  Change: {reweighted_dp_diff - baseline_dp_diff:.4f}")

print(f"\nDisparate Impact:")
print(f"  Before: {dataset_metric.disparate_impact():.4f}")
print(f"  After:  {dataset_transf_metric.disparate_impact():.4f}")
print(f"  Change: {dataset_transf_metric.disparate_impact() - dataset_metric.disparate_impact():.4f}")



FAIRNESS COMPARISON: BEFORE vs AFTER MITIGATION

MODEL PERFORMANCE:
Baseline Accuracy: 0.8007
Reweighted Accuracy: 0.8007
Accuracy Change: 0.0000

FAIRNESS METRICS:
Demographic Parity Difference:
  Before: -0.1963
  After:  0.0000
  Change: 0.1963

Disparate Impact:
  Before: 0.3580
  After:  1.0000
  Change: 0.6420


In [15]:
# Summary table of key metrics
summary_data = {
    'Metric': [
        'Overall Accuracy',
        'Demographic Parity Difference',
        'Disparate Impact'
    ],
    'Before Mitigation': [
        f"{baseline_accuracy:.4f}",
        f"{baseline_dp_diff:.4f}",
        f"{dataset_metric.disparate_impact():.4f}"
    ],
    'After Mitigation': [
        f"{accuracy_transf:.4f}",
        f"{reweighted_dp_diff:.4f}",
        f"{dataset_transf_metric.disparate_impact():.4f}"
    ],
    'Change': [
        f"{accuracy_transf - baseline_accuracy:+.4f}",
        f"{reweighted_dp_diff - baseline_dp_diff:+.4f}",
        f"{dataset_transf_metric.disparate_impact() - dataset_metric.disparate_impact():+.4f}"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\nSUMMARY OF RESULTS:")
print(summary_df.to_string(index=False))

# Interpretation
print("\n" + "="*60)
print("INTERPRETATION")
print("="*60)

print("\n1. ACCURACY IMPACT:")
if accuracy_transf > baseline_accuracy:
    print(f"   ✓ Reweighting improved accuracy by {accuracy_transf - baseline_accuracy:.4f}")
elif accuracy_transf < baseline_accuracy:
    print(f"   ✗ Reweighting decreased accuracy by {baseline_accuracy - accuracy_transf:.4f}")
else:
    print("   → Accuracy remained unchanged")

print("\n2. FAIRNESS IMPROVEMENTS:")
if reweighted_dp_diff < baseline_dp_diff:
    print(f"   ✓ Demographic parity improved by {baseline_dp_diff - reweighted_dp_diff:.4f}")
else:
    print(f"   ✗ Demographic parity worsened by {reweighted_dp_diff - baseline_dp_diff:.4f}")
    
disparate_impact_before = dataset_metric.disparate_impact()
disparate_impact_after = dataset_transf_metric.disparate_impact()
if abs(disparate_impact_after - 1.0) < abs(disparate_impact_before - 1.0):
    print(f"   ✓ Disparate impact improved (closer to 1.0)")
else:
    print(f"   ✗ Disparate impact worsened")

print("\n3. KEY INSIGHTS:")
print("   • AIF360 provides standardized fairness metrics")
print("   • Reweighting technique aims to balance demographic representation")
print("   • Trade-offs between fairness and accuracy are common")
print("   • Multiple fairness metrics may conflict with each other")



SUMMARY OF RESULTS:
                       Metric Before Mitigation After Mitigation  Change
             Overall Accuracy            0.8007           0.8007 +0.0000
Demographic Parity Difference           -0.1963           0.0000 +0.1963
             Disparate Impact            0.3580           1.0000 +0.6420

INTERPRETATION

1. ACCURACY IMPACT:
   → Accuracy remained unchanged

2. FAIRNESS IMPROVEMENTS:
   ✗ Demographic parity worsened by 0.1963
   ✓ Disparate impact improved (closer to 1.0)

3. KEY INSIGHTS:
   • AIF360 provides standardized fairness metrics
   • Reweighting technique aims to balance demographic representation
   • Trade-offs between fairness and accuracy are common
   • Multiple fairness metrics may conflict with each other
